This notebook demonstrates how to use Quest to download imagery from a Web Map Tile Service (WMTS).

In addition to quest the following packages need to be installed to use this notebook:
 * holoviews
 * geoviews
 * param
 * paramnb
 * xarray
 
The can be installed with the following command:
```
conda install -c conda-forge -c pyviz/label/dev holoviews geoviews param paramnb xarray
```

In [ ]:
import param
import quest
import geoviews as gv
import holoviews as hv
import xarray as xr
from cartopy import crs as ccrs
from holoviews.streams import BoxEdit
from parambokeh import Widgets

hv.extension('bokeh')

In [ ]:
quest_service = 'svc://wmts:seamless_imagery'
tile_service_options = quest.api.get_download_options(quest_service, fmt='param')[quest_service]
tile_service_options.params()['bbox'].precedence = -1  # hide bbox input
Widgets(tile_service_options)

In [ ]:
tiles = gv.WMTS(tile_service_options.url).options(width=950, height=600, global_extent=True)
boxes = gv.Polygons([]).options(fill_alpha=0.4, line_width=2)
box_stream = BoxEdit(source=boxes, num_objects=1)
tiles * boxes

In [ ]:
if box_stream.element:
    data = box_stream.data
    bbox = [data['x0'][0], data['y0'][0], data['x1'][0], data['y1'][0]]
else:
    bbox = [-72.43925984610391, 45.8471360126193, -68.81252476472281, 47.856449699679516]
#     bbox = [-83.28613682601174, 24.206059963486737, -81.93264005405752, 30.251169660148314]
#     bbox = [-81.82408198648336, 25.227665888548458, -80.86355086047537, 31.548730116206755]
print(bbox)
tile_service_options.bbox = bbox

In [ ]:
arr = quest.api.get_data(
    service_uri=quest_service,
    search_filters=None,
    download_options=tile_service_options,
    collection_name='examples',
    use_cache=False,
    as_open_datasets=True,
)[0]

In [ ]:
image = gv.RGB((arr.x, arr.y, arr[0].values, 
        arr[1].values, arr[2].values), 
       vdims=['R', 'G', 'B']).options(width=950, height=600, alpha=0.7)
gv.tile_sources.Wikipedia * image